In [ ]:
import pandas as pd
from transformers import pipeline

In [ ]:
! pip -q install transformers

In [ ]:
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).Using a pipeline without specifying a model name and revision in production is not recommended./usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: The secret `HF_TOKEN` does not exist in your Colab secrets.To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.You will be able to reuse this secret in all of your notebooks.Please note that authentication is recommended but still optional to access public models or datasets.  warnings.warn(

config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()  return self.fget.__get__(instance, owner)()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv('/kaggle/input/mirror/Mirror_label_dataset_using_hybrid_model (1).csv')
df.rename(columns={'summary': 'article'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>RangeIndex: 1334 entries, 0 to 1333Data columns (total 3 columns): #   Column   Non-Null Count  Dtype ---  ------   --------------  -----  0   Date     1334 non-null   object 1   Title    1334 non-null   object 2   Article  1322 non-null   objectdtypes: object(3)memory usage: 31.4+ KB

In [ ]:
d = df[700:1000]
max_length = 3000
d['truncated_text'] = d['Article'].str.slice(0, max_length)
articles = d['truncated_text'].tolist()

<ipython-input-27-535b7f698c58>:3: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.Try using .loc[row_indexer,col_indexer] = value insteadSee the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy  d['truncated_text'] = d['Article'].str.slice(0, max_length)

In [ ]:
summaries = []
for article in articles:
    if pd.notnull(article):
        summary = summarizer(article)[0]['summary_text']
        summaries.append(summary)
    else:
        summaries.append(None)
d['summary'] = summaries
d.to_csv('Mirror_news_summaries(for_hybrid_ML)_700_to1000.csv', index=False)

<ipython-input-28-fae5e66b6d92>:8: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.Try using .loc[row_indexer,col_indexer] = value insteadSee the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy  d['summary'] = summaries

In [ ]:
data = pd.read_csv('/content/Mirror_news_summaries(for_hybrid_ML)_350.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>RangeIndex: 350 entries, 0 to 349Data columns (total 5 columns): #   Column          Non-Null Count  Dtype ---  ------          --------------  -----  0   Date            350 non-null    object 1   Title           350 non-null    object 2   Article         338 non-null    object 3   truncated_text  338 non-null    object 4   summary         338 non-null    objectdtypes: object(5)memory usage: 13.8+ KB

# Summarization T5

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:160: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.  warnings.warn(/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language model

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Guardian_society_Final_Dataset_with_Articles.csv')
d = df[:10]
max_length = 3000
d['truncated_text'] = d['Article'].str.slice(0, max_length)
articles = d['truncated_text'].tolist()

<ipython-input-8-90d728e37609>:4: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.Try using .loc[row_indexer,col_indexer] = value insteadSee the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy  d['truncated_text'] = d['Article'].str.slice(0, max_length)

In [ ]:
summaries = []
for article in articles:
    if pd.notnull(article):
        inputs = tokenizer.encode("summarize: " + article, return_tensors='pt', max_length=512, truncation=True)
        summary_ids = model.generate(inputs, max_length=150, min_length=80, length_penalty=5., num_beams=2)
        summary = tokenizer.decode(summary_ids[0])
        summaries.append(summary)
    else:
        summaries.append(None)
d['summary'] = summaries
d.to_csv('news_summ.csv', index=False)

<ipython-input-9-c9d0e898fda5>:10: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.Try using .loc[row_indexer,col_indexer] = value insteadSee the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy  d['summary'] = summaries